<a href="https://colab.research.google.com/github/marco-siino/SemEval2024/blob/main/Task%205/SemEval_2024_Task_5_mistral_instruct_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 238.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 310.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.37-cp310-cp310-manylinux_2_35_x86_64.whl size=9593134 sha256=449a697ecd396812372a41722cc125062bcb25ab08acbbbf153c319e9675b79d
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7ct8mc7/wheels/5b/4f/36/1df917d3d21e5d04ce960616a6fb9e39c1548149620528a67c
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.

We start by downloading an instruction-finetuned Mistral model, which we will ask to classify model outputs for us.

In [2]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

#model_name_or_path='TheBloke/Llama-2-7B-Chat-GGUF'
#model_basename = "llama-2-7b-chat.Q2_K.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

run_on_test = False # whether this baseline system is ran on the test splits or the val splits

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '32768', 'general.na

In [3]:
run_on_test = True # whether this baseline system is ran on the test splits or the val splits

# Import test set from GitHub

In [4]:
!git clone https://github.com/marco-siino/SemEval2024.git

Cloning into 'SemEval2024'...
fatal: could not read Username for 'https://github.com': No such device or address


# Convert CSV test file into json.

In [5]:
import csv
import json

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []

    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf:
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf)

        #convert each csv row into python dict
        for row in csvReader:
            #add this python dict to json array
            jsonArray.append(row)

    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)

csvFilePath = r'test.csv'
jsonFilePath = r'test.json'
csv_to_json(csvFilePath, jsonFilePath)

Running on the test data

In [6]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
test_set = "test.json"
results = "results.csv"

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
# Import pandas library
import pandas as pd
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# Create the pandas DataFrame with the results.
df_result = pd.DataFrame(columns=['idx', 'baseline'])

# simple JSON loading
with open(test_set, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)

output_json = []
labels = [0,1]
#labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Question: {}
Answer: {}
Is the answer for the question above true or false?
Answer True or False:
"""
for i in tqdm.trange(num_sample):
    #task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['idx'])
    question = str(data_val_all[i]['question'])
    answer = str(data_val_all[i]['answer'])
    #tgt = str(data_val_all[i]['tgt'])

    #if task == "PG":
        #context = f"Context: {src}"
    #else: #i.e. task == "MT" or task == "DM":
        #context = f"Context: {tgt}"
    context = f"Question: {question}"

    sentence = f"Answer: {answer}"
    message = f"{context}\n{sentence}\nIs the Answer to the Question above True or False? Answer using ONLY True or False:"
    prompt = f"<s>[INST] {message} [/INST]"
    print("\n",prompt)

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    print(answer)
    if answer.startswith("true"):
        output_label = 1
        print(output_label)
        #prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("false"):
        output_label = 0
        print(output_label)
        #prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("true") and not answer.startswith("false"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        #prob = float(0.5)

    df_result.loc[len(df_result.index)] = [id, output_label]

98


  0%|          | 0/98 [00:00<?, ?it/s]


 <s>[INST] Question: 8. International incident. Tiant sues Lenoir, who lives in the Southern District of New York, and Sorrel, who lives in the Western District of Pennsylvania, for negligence in causing an auto accident in the province of Ontario, Canada. Tiant, a pedestrian from Ontario, was injured when cars driven by Lenoir and Sorrel collided and swerved off the road. He alleges that Lenoir was negligent in running a red light at the time of the accident, causing the collision with Sorrel. He claims that Sorrel was negligent in improperly repairing his brakes himself at his home in Pennsylvania, so he couldn’t avoid hitting Tiant after the collision. He brings suit in the federal district court for the Southern District of New York, and serves Sorrel with process in the action while Sorrel is visiting New York City.
Answer: Venue is proper in the Southern District of New York under 28 U.S.C. §1391(b)(3) since the defendants are subject to personal jurisdiction in New York.
Is the

Llama.generate: prefix-match hit


false. the answer is not correct. the venue is proper in the southern district
0

 <s>[INST] Question: 8. International incident. Tiant sues Lenoir, who lives in the Southern District of New York, and Sorrel, who lives in the Western District of Pennsylvania, for negligence in causing an auto accident in the province of Ontario, Canada. Tiant, a pedestrian from Ontario, was injured when cars driven by Lenoir and Sorrel collided and swerved off the road. He alleges that Lenoir was negligent in running a red light at the time of the accident, causing the collision with Sorrel. He claims that Sorrel was negligent in improperly repairing his brakes himself at his home in Pennsylvania, so he couldn’t avoid hitting Tiant after the collision. He brings suit in the federal district court for the Southern District of New York, and serves Sorrel with process in the action while Sorrel is visiting New York City.
Answer: Venue is not proper in the Southern District of New York; it is only proper i

...

false. the answer provided is incorrect as the service of process and personal jurisdiction
0

 <s>[INST] Question: 9. Problems, problems. Preferred Metalworks, Inc. is incorporated in Delaware and has its principal place of business in Kentucky. It also has a factory in Ohio, where Stamski worked. One day Donitz, president of Preferred, called Stamski into his office at the Ohio plant and fired him. Stamski sues Preferred and Donitz in federal court for breach of contract. In which of the following situations is service of process proper on both defendants and is personal jurisdiction proper over both defendants?
Answer: Stamski sues in federal court in Kentucky. He serves Preferred by mailing the summons and complaint, first class mail, to Prosnitz, secretary of the corporation, at her home in Ohio. Ohio state law authorizes service on a corporation by mail to a corporate officer, but Kentucky law does not. He serves Donitz by having a process server deliver his summons and the compl

Llama.generate: prefix-match hit


false. the answer provided is incorrect regarding service of process on preferred metalworks
0

 <s>[INST] Question: 9. Problems, problems. Preferred Metalworks, Inc. is incorporated in Delaware and has its principal place of business in Kentucky. It also has a factory in Ohio, where Stamski worked. One day Donitz, president of Preferred, called Stamski into his office at the Ohio plant and fired him. Stamski sues Preferred and Donitz in federal court for breach of contract. In which of the following situations is service of process proper on both defendants and is personal jurisdiction proper over both defendants?
Answer: Service and personal jurisdiction are not both proper in any of the cases above.
Is the Answer to the Question above True or False? Answer using ONLY True or False: [/INST]


Llama.generate: prefix-match hit


false. the question asks for situations where both service and personal jurisdiction are proper
0


In [13]:
import zipfile

df_result.to_csv('result.csv',index=False)
zip = zipfile.ZipFile('submission_result.zip', 'w', zipfile.ZIP_DEFLATED)
zip.write('result.csv')
zip.close()

In [ ]:
from google.colab import files
files.download('submission_result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>